In [ ]:
testRep = 50

resultOLS = pd.DataFrame(index = range(testRep), columns = ['In-sample profit (OLS)', 'In-sample profit (Integrated)', 
                                                            'Out-of-sample profit (OLS)', 'Out-of-sample profit (Integrated)'])

for i in range(testRep):
    print('Test run = ', i)
    coef1, coef2 = samplingDemand(sampleSize=n, coefVar=cv)
    coef2=coef2[0]
    testU = outOfSampleError(testSize = 100000)

    optV, optP, optQ, optVDD, optPDD, optQDD = DDNV_main(sampleSize=n, procurementCost=c, 
                                                         estimationModel = 'cost', nonStockOut = None, priceUpperBound = ubp)

    (optV_OLS, optP_OLS, optQ_OLS) = NV_Price(procurementCost = c, beta0 = coef1, beta1 = coef2, priceUpperBound = ubp)
    
    (optVDD_OLS, optPDD_OLS, optQDD_OLS) = DDNV_Price(sampleSize = n, procurementCost = c, 
                                                      beta0 = coef1, beta1 = coef2, priceUpperBound = ubp)
    
    testD = outOfSampleDemand(testU, chosenPrice = optP)
    testV = testEvaluation(testD, chosenPrice = optP, chosenQty = optQ, procurementCost = c)

    #Data-driven approach
    testD_OLS = outOfSampleDemand(testU, chosenPrice = optP_OLS)
    testV_OLS = testEvaluation(testD_OLS, chosenPrice = optP_OLS, chosenQty = optQ_OLS, procurementCost = c)

    resultOLS.iloc[i,:] = [optV_OLS, optV, testV_OLS, testV]
print(resultOLS)

In [ ]:
print('Avg. in-sample profit (OLS)', np.mean(resultOLS.iloc[:,0]))
print('Avg. in-sample profit (Integrated)', np.mean(resultOLS.iloc[:,1]))
print('Avg. out-of-sample profit (OLS)', np.mean(resultOLS.iloc[:,2]))
print('Avg. out-of-sample profit (Integrated)', np.mean(resultOLS.iloc[:,3]))

In [ ]:
testRep = 50

resultOLS1 = pd.DataFrame(index = range(testRep), columns = ['Avg. in-sample profit (OLS)', 'Avg. in-sample profit (Integrated)', 'Avg. out-of-sample profit (OLS)', 'Avg. out-of-sample profit (Integrated)'])

for i in range(testRep):
    print('Test run = ', i)
    coef1, coef2 = samplingDemand(sampleSize=n, coefVar=cv)
    coef2=coef2[0]
    testU = outOfSampleError(testSize = 100000)

    optV, optP, optQ, optVDD, optPDD, optQDD = DDNV_main(sampleSize=n, procurementCost=c, 
                                                         estimationModel = 'cost', nonStockOut = None, priceUpperBound = ubp)

    (optV_OLS, optP_OLS, optQ_OLS) = NV_Price(procurementCost = c, beta0 = coef1, beta1 = coef2, priceUpperBound = ubp)
    
    (optVDD_OLS, optPDD_OLS, optQDD_OLS) = DDNV_Price(sampleSize = n, procurementCost = c, 
                                                      beta0 = coef1, beta1 = coef2, priceUpperBound = ubp)
    
    testD = outOfSampleDemand(testU, chosenPrice = optPDD)
    testV = testEvaluation(testD, chosenPrice = optPDD, chosenQty = optQDD, procurementCost = c)

    #Data-driven approach
    testD_OLS = outOfSampleDemand(testU, chosenPrice = optPDD_OLS)
    testV_OLS = testEvaluation(testD_OLS, chosenPrice = optPDD_OLS, chosenQty = optQDD_OLS, procurementCost = c)

    resultOLS1.iloc[i,:] = [optV_OLS, optV, testV_OLS, testV]
print(resultOLS1)

In [ ]:
print('Avg. in-sample profit (OLS)', np.mean(resultOLS1.iloc[:,0]))
print('Avg. in-sample profit (Integrated)', np.mean(resultOLS1.iloc[:,1]))
print('Avg. out-of-sample profit (OLS)', np.mean(resultOLS1.iloc[:,2]))
print('Avg. out-of-sample profit (Integrated)', np.mean(resultOLS1.iloc[:,3]))

In [ ]:
3500068914, 4116709428, 4116709565, 4116709448, 2066200532, 3700031613

In [ ]:
2066200532

In [ ]:
import copy

testRep = 20

# Run the model with different sample sizes: 100, 200, 500 and 700
sampleSizeSet = [50, 100, 200, 500, 700]
resultSize = pd.DataFrame(index = range(len(sampleSizeSet)), columns = ['Sample size', 'Avg. out-of-sample profit'])
optValOutSample = np.zeros(shape=(len(sampleSizeSet),testRep))

for j in range(testRep):
    print('Test run = ', j)
    samplingDemand(sampleSize=max(sampleSizeSet), coefVar=cv)
    demandPool = copy.deepcopy(D)
    pricePool = copy.deepcopy(p)
    errorPool = copy.deepcopy(u)
    
    testU = outOfSampleError(testSize = 100000)
    
    for i in range(len(sampleSizeSet)):
        print('Considering n =', sampleSizeSet[i])
        if sampleSizeSet[i] != max(sampleSizeSet):
            lowElasticity = True
            while lowElasticity:
                idx = np.random.choice(range(max(sampleSizeSet)), size = sampleSizeSet[i], replace = False)
                D = demandPool[idx]; p = pricePool[idx]; u = errorPool[idx];
                
                reg = linear_model.LinearRegression().fit(p, D)
                coef2 = reg.coef_
                coef1 = reg.intercept_
                Q1 = coef1 + coef2*min(p)
                Q2 = coef1 + coef2*max(p)
                arcElasticity = ( (min(p) + max(p)) / (Q1 + Q2)) * ((Q2 - Q1) / (max(p) - min(p)) )
                if abs(arcElasticity) >= 1: lowElasticity = False

        else:
            D = demandPool; p = pricePool; u = errorPool;
        
        optV, optP, optQ, optVDD, optPDD, optQDD = DDNV_main(sampleSize=sampleSizeSet[i], procurementCost=c, 
                                                             estimationModel = 'service', nonStockOut = alpha, priceUpperBound=ubp)
        
        testD = outOfSampleDemand(testU, chosenPrice = optPDD)
        testV = testEvaluation(testD, chosenPrice = optPDD, chosenQty = optQDD, procurementCost = c)
        
        optValOutSample[i,j] = testV
        
for i in range(len(sampleSizeSet)):
    resultSize.iloc[i,:] = [sampleSizeSet[i], np.mean(optValOutSample[i,:])]